### Install required packages and import them

In [ ]:
!pip install keybert[all]
!pip install -U sentence-transformers

In [ ]:
import pandas as pd
from keybert import KeyBERT
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

### Import CSV File from server 

In [ ]:
df = pd.read_csv("insert path name here", delimiter = '$', encoding = 'utf-8-sig', engine ='python')

### Filter out junk comments

filters out all comments with a character length of 4 or less (e.g. "n.a." removed, which was causing a problem with the application)

In [ ]:
df['ERA_QuestionComment'] = df['ERA_QuestionComment'].astype('str')
mask = (df['ERA_QuestionComment'].str.len() > 4)
df = df.loc[mask]
df.head(10)

### Define main function and loop it through all comments

Defines the function 'Keywords' that take an input x (the comment) and runs the keyword extraction on it:

In [ ]:
def Keywords(x):
    doc = x
    kw_model = KeyBERT(model)
    keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None, highlight=True, top_n=10)
    print(keywords)
    return keywords

Loops all comments through the function and puts them into a new column 'ER_Keywords':

In [ ]:
df["ER_Keyword"] = df["ERA_QuestionComment"].apply(Keywords)

### Clean the output

takes each keyword and put it into its own row:

In [ ]:
df = df.explode('ER_Keyword', ignore_index=True)
df = df.explode('ER_Keyword', ignore_index=True)

Deltetes every other row (ever other row contains the relevance score - note needed):

In [ ]:
df = df.iloc[::2]

In [ ]:
df.head(20)

Removes all (most likely non-relevant) keywords that are 3 characters or less:

In [ ]:
df['ER_Keyword'] = df['ER_Keyword'].astype('str')
mask = (df['ER_Keyword'].str.len() >= 4)
df = df.loc[mask]

In [ ]:
df.head(30)

### Export table as CSV 

In [ ]:
df.to_csv('replace desired file path here', index = False, encoding = 'utf-8-sig')